In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
! pip install -U spacy -q

In [3]:
!python -m spacy info


============================== Info about spaCy ==============================

spaCy version    3.3.0                         
Location         /usr/local/lib/python3.7/dist-packages/spacy
Platform         Linux-5.4.188+-x86_64-with-Ubuntu-18.04-bionic
Python version   3.7.13                        
Pipelines                                      



In [4]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object

In [5]:
import json
filename = '/content/drive/MyDrive/Custom-NER/Datasets/annotations.json'
f = open(filename)
TRAIN_DATA = json.load(f)

In [6]:
TRAIN_DATA

{'annotations': [['2021-07-08 13:45:57 INFO WMQJCAResourceAdapter: 0] =>>>GenericMessageDrivenBean - start() <Envelope><Header><CommonInfo><Source>OTP</Source><Service>SMSSrv</Service></CommonInfo>\r\n</Header><Body><SMSSrv><Mobile><CountryCode>61</CountryCode><Num>0420742903</Num>\r\n</Mobile><Source>IBNK</Source><RefNum>6266342</RefNum><RefNum2>1</RefNum2>\r\n<Bank>STG</Bank><GCISCustNum>11181855</GCISCustNum><SMSMsg>Your Secure Code is 494244 to add\r\nPayee with BB:062-013 Account:424242454. Never share this code.</SMSMsg></SMSSrv></Body></Envelope>\r\n(au.com.stgeorge.sms.integration.impl.listener.GenericMessageDrivenBean)\r\n',
   {'entities': [[418, 424, 'SOURCECODE']]}],
  ['=\r\n2021-07-02 15:06:41:806 INFO [WMQJCAResourceAdapter: 29] =>getAlertsByAccountNumber START Alert ID:0\r\nApp ID :CRA Acc Num :4601841000532921 Acc Type :null Type :0 GCIS null\r\n(au.com.stgeorge.sms.business.ProfileManagerlmpl)\r\n',
   {'entities': [[125, 142, 'ACCNUM']]}],
  ['=\r\n2021-07-08 13:45:5

In [7]:
for text, annot in tqdm(TRAIN_DATA['annotations']): 
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents 
    db.add(doc)

db.to_disk("/content/drive/MyDrive/Custom-NER/Datasets/training_data.spacy") # save the docbin object

100%|██████████| 4/4 [00:00<00:00, 404.74it/s]

Skipping entity


In [8]:
pip install spacy-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
! python -m spacy init config /content/drive/MyDrive/Custom-NER/Datasets/config.cfg --lang en --pipeline ner --optimize efficiency

ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
/content/drive/MyDrive/Custom-NER/Datasets/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [10]:
! python -m spacy train /content/drive/MyDrive/Custom-NER/Datasets/config.cfg --output /content/drive/MyDrive/Custom-NER/Models --paths.train /content/drive/MyDrive/Custom-NER/Datasets/training_data.spacy --paths.dev /content/drive/MyDrive/Custom-NER/Datasets/training_data.spacy

ℹ Saving to output directory:
/content/drive/MyDrive/Custom-NER/Models
ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

=========================== Initializing pipeline ===========================
[2022-05-26 11:12:33,525] [INFO] Set up nlp object from config
[2022-05-26 11:12:33,535] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-05-26 11:12:33,539] [INFO] Created vocabulary
[2022-05-26 11:12:33,540] [INFO] Finished initializing nlp object
[2022-05-26 11:12:33,675] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     35.73    0.00    0.00    0.00    0.00
 57     200         88.03    979.93   76.92   71.43   83.33    0.77
132     400         10.19   

In [11]:
nlp_ner = spacy.load("/content/drive/MyDrive/Custom-NER/Models/model-best")

In [15]:
text = '''2021-07-08 13:45:57 INFO WMQJCAResourceAdapter: 0] =>>>GenericMessageDrivenBean - start()<Envelope><Header><CommonInfo><Source>OTP</Source><Service>SMSSrv</Service></CommonInfo>
</Header><Body><SMSSrv><Mobile><CountryCode>61</CountryCode><Num>0420742903</Num>
</Mobile><Source>IBNK</Source><RefNum>6266342</RefNum><RefNum2>1</RefNum2><Acctkey>1234567890987654</Acctkey>
<Bank>STG</Bank><GCISCustNum>11181855</GCISCustNum><SMSMsg>Your Secure Code is 494244 to add
Payee with BB:062-013 Account:424242454. Never share this code.</SMSMsg></SMSSrv></Body></Envelope>
(au.com.stgeorge.sms.integration.impl.listener.GenericMessageDrivenBean)'''

In [16]:
doc = nlp_ner(text)

In [17]:
spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter